In [6]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Define the base directory where the data is stored
base_dir = "/content/MinuteRaw15/MinuteRaw15"  # Update to the exact path where year folders are located

# Step 2: Initialize an empty DataFrame to combine all data
combined_df = pd.DataFrame()

# Step 3: Loop through each year folder
for year in range(2012, 2024):
    year_dir = os.path.join(base_dir, str(year))  # Path to the year folder
    if not os.path.exists(year_dir):  # Check if the directory exists
        print(f"Year directory not found: {year_dir}")
        continue

    # Get the sorted list of .csv files for each year
    csv_files = sorted([f for f in os.listdir(year_dir) if f.endswith('.csv')])

    for file in csv_files:
        file_path = os.path.join(year_dir, file)
        monthly_df = pd.read_csv(file_path)  # Read each CSV file
        combined_df = pd.concat([combined_df, monthly_df], ignore_index=True)  # Concatenate the data

# Step 4: Create the 'date' column
start_time = datetime(2012, 1, 1, 0, 15)
time_intervals = [start_time + timedelta(minutes=15 * i) for i in range(len(combined_df))]
combined_df['date'] = time_intervals

# Step 5: Save the combined dataset
output_file = "/content/combined_time_series.csv"
combined_df.to_csv(output_file, index=False)

print(f"Combined dataset saved to {output_file}")


Combined dataset saved to /content/combined_time_series.csv


In [7]:
12*365*24*4

420480

In [ ]:
from google.colab import files
import os
import pandas as pd
from datetime import datetime, timedelta
import zipfile

# Step 1: Upload and unzip the folder
uploaded = files.upload()

zip_file = "/content/MinuteRaw15.zip"
extract_dir = "/content/MinuteRaw15"
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Step 2: Dynamically find the correct base directory
for root, dirs, files in os.walk("/content"):
    if "2012" in dirs:
        base_dir = os.path.join(root, "MinuteRaw15", "MinuteRaw15")  # Adjusted to match the actual path
        print(f"Base directory set to: {base_dir}")
        break


# Step 3: Combine files
combined_df = pd.DataFrame()

for year in range(2012, 2024):
    year_dir = os.path.join(base_dir, str(year))  # Path to the year folder
    if not os.path.exists(year_dir):  # Check if the directory exists
        print(f"Year directory not found: {year_dir}")
        continue

    csv_files = sorted([f for f in os.listdir(year_dir) if f.endswith('.csv')])

    for file in csv_files:
        file_path = os.path.join(year_dir, file)
        monthly_df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, monthly_df], ignore_index=True)

# Step 4: Create the 'date' column
start_time = datetime(2012, 1, 1, 0, 15)
time_intervals = [start_time + timedelta(minutes=15 * i) for i in range(len(combined_df))]
combined_df['date'] = time_intervals

# Step 5: Save the combined dataset
output_file = "/content/combined_time_series.csv"
combined_df.to_csv(output_file, index=False)

print(f"Combined dataset saved to {output_file}")


In [ ]:
# IN ORDER TO CHECK THE TIME PROGRESSION

import pandas as pd

# Load the combined dataset
file_path = "/content/combined_time_series.csv"
df = pd.read_csv(file_path)

# Check the start and end of the dataset
print("Start date:", df['date'].iloc[0])
print("End date:", df['date'].iloc[-1])

# Verify that intervals are consistent
df['date'] = pd.to_datetime(df['date'])
time_differences = df['date'].diff().dropna().value_counts()

print("Unique time intervals in the dataset:", time_differences)


In [ ]:
# CHECKING THE MONTHLY CONCATENATION

# Extract year and month from the 'date' column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Group by year and month and count rows
month_counts = df.groupby(['year', 'month']).size().reset_index(name='row_count')

# Display the first few rows and the total counts
print(month_counts.head(12))  # Display data for 2012
print(month_counts.tail(12))  # Display data for 2023


In [ ]:
# CHECKING DATA FROM SPECIFIC MONTHS

# Example: Check January 2012
jan_12 = df[df['date'].dt.to_period('M') == '2012-01']
print("January 2012 rows:", len(jan_12))
print("First timestamp in January 2012:", jan_12['date'].iloc[0])
print("Last timestamp in January 2012:", jan_12['date'].iloc[-1])

# Example: Check December 2023
dec_23 = df[df['date'].dt.to_period('M') == '2023-12']
print("December 2023 rows:", len(dec_23))
print("First timestamp in December 2023:", dec_23['date'].iloc[0])
print("Last timestamp in December 2023:", dec_23['date'].iloc[-1])


In [ ]:
# CONFIRM THE TOTAL ROWS

# Calculate expected row count
start = pd.Timestamp("2012-01-01 00:15")
end = pd.Timestamp("2023-12-31 00:00")
expected_rows = int(((end - start).total_seconds() / 60) / 15) + 1

print("Expected row count:", expected_rows)
print("Actual row count:", len(df))


In [8]:
# CONVERTING TO AN HOURLY DATA SET

import pandas as pd
from datetime import datetime, timedelta

# Step 1: Load the dataset without the 'date' column
file_path = "/content/combined_time_series.csv"
df = pd.read_csv(file_path)
df = df.drop(columns=['date'])  # Remove the date column

# Step 2: Sum every 4 consecutive rows to create hourly data
# Reshape the data by summing over every 4 rows
hourly_df = df.groupby(df.index // 4).sum()

# Step 3: Generate a new hourly date column
start_time = datetime(2012, 1, 1, 1, 0)  # Start at the first hour of Jan 1, 2012
hourly_dates = [start_time + timedelta(hours=i) for i in range(len(hourly_df))]
hourly_df['date'] = hourly_dates

# Step 4: Save the resulting hourly dataset
output_file = "/content/hourly_time_series.csv"
hourly_df.to_csv(output_file, index=False)

print(f"Hourly dataset saved to {output_file}")


Hourly dataset saved to /content/hourly_time_series.csv


In [ ]:
# VERIFICATION 1 (start and end date)

print("Start date:", hourly_df['date'].iloc[0])
print("End date:", hourly_df['date'].iloc[-1])


In [ ]:
# VERIFICATION 2 (number of rows in the dataset)

print("Number of rows in hourly dataset:", len(hourly_df))


In [9]:
# CREATING HOURLY DATA SETS

import pandas as pd

# Step 1: Load the hourly dataset
file_path = "/content/hourly_time_series.csv"
df = pd.read_csv(file_path)

# Convert the 'date' column to datetime for filtering by time
df['date'] = pd.to_datetime(df['date'])

# Step 2: Define the target hours
target_hours = [8, 14, 19, 0]  # 8 AM, 2 PM, 7 PM, 12 AM

# Step 3: Filter the data for each target hour
filtered_datasets = {}
for hour in target_hours:
    filtered_df = df[df['date'].dt.hour == hour].reset_index(drop=True)
    filtered_datasets[hour] = filtered_df

    # Save the filtered dataset
    output_file = f"/content/daily_data_{hour:02d}.csv"
    filtered_df.to_csv(output_file, index=False)
    print(f"Dataset for {hour:02d}:00 saved to {output_file}")


Dataset for 08:00 saved to /content/daily_data_08.csv
Dataset for 14:00 saved to /content/daily_data_14.csv
Dataset for 19:00 saved to /content/daily_data_19.csv
Dataset for 00:00 saved to /content/daily_data_00.csv


In [ ]:
# VERIFICATION 1 checking the row count

for hour, data in filtered_datasets.items():
    print(f"Hour {hour:02d}: Number of rows = {len(data)}")


In [ ]:
# VERIFICATION 2 preview the data

print(filtered_datasets[8].head())
print(filtered_datasets[8].tail())


In [ ]:
# VERIFICATION 3 checking the date column

for hour, data in filtered_datasets.items():
    print(f"Unique hours for {hour:02d} dataset:", data['date'].dt.hour.unique())
